In [1]:
#import data processing libraries

import os
import pandas as pd
import numpy as np
import math as math
import datetime as dt
from scipy.stats import mode
import psycopg2 #reading data from Postgres DB
import sys
import openpyxl

In [2]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

#suppress scientific notation to 2 decimal places
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
#import visualization libraries

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

In [4]:
# Connection parameters
param_dic = {
    "host"      : "157.245.248.249",
    "database"  : "ubuntu",
    "user"      : "jacklinengenia",
    "password"  : "x3MX&8#!"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [5]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # Convert the list of tuples in to a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [6]:
# Connect to the database and load the scoring data bloomlive table
conn = connect(param_dic)

columns = "client_mifos_id,client_mobile_number,loan_status,loan_mifos_id,term_frequency,principal_disbursed,\
principal_repaid,interest_charged,interest_repaid,\
fee_charges_charged,fee_charges_repaid,\
penalty_charges_charged,penalty_charges_repaid,\
total_outstanding,\
disbursed_on_date,expected_matured_on_date,\
closed_on_date,store_number, bloom_version"

column_names = columns.strip().split(",")
# Execute the "SELECT cols" query
df = postgresql_to_dataframe(conn,
                             "select \
                             client_mifos_id,client_mobile_number,loan_status,loan_mifos_id,term_frequency,principal_disbursed,\
                             principal_repaid,interest_charged,interest_repaid,\
                             fee_charges_charged,fee_charges_repaid,\
                             penalty_charges_charged,penalty_charges_repaid,\
                             total_outstanding,\
                             disbursed_on_date,expected_matured_on_date,\
                             closed_on_date,store_number, bloom_version\
                             from bloomlive.loans_fact_table_materialized_summary_view",
                             column_names)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
#replace None type datatype with Nan values across entire df

df = df.fillna(value=np.nan)

In [8]:
#convert date columns
datetime_cols = ["disbursed_on_date", "expected_matured_on_date", "closed_on_date"]

df[datetime_cols] = df[datetime_cols].apply(pd.to_datetime, errors='coerce')


#convert specific int/float columns to string type
int_to_string_cols = ["client_mifos_id", "client_mobile_number"]

df[int_to_string_cols] = df[int_to_string_cols].astype(str)


#convert specific string columns to float, first by leaving out the non-targeted columns
string_to_float_cols = df.columns.drop(["client_mifos_id", "loan_status", "term_frequency",
                                        "disbursed_on_date", "expected_matured_on_date",
                                        "closed_on_date"])

df[string_to_float_cols] = df[string_to_float_cols].apply(pd.to_numeric, errors='coerce')


#replace the in the column headers with underscores & make all column headers in lower case
df.columns = df.columns.str.lower().str.replace(' ','')

In [9]:
#create new column that concats the bloom version and loan id so as to prevent any loan duplicates from the same mifos instance

df["loan_id_product_concat"] = (df["loan_mifos_id"].astype("str")+"-"+df["bloom_version"].astype("str")).astype("str")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434298 entries, 0 to 434297
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   client_mifos_id           434298 non-null  object        
 1   client_mobile_number      105067 non-null  float64       
 2   loan_status               434298 non-null  int64         
 3   loan_mifos_id             434298 non-null  int64         
 4   term_frequency            434298 non-null  int64         
 5   principal_disbursed       434298 non-null  float64       
 6   principal_repaid          434298 non-null  float64       
 7   interest_charged          434298 non-null  float64       
 8   interest_repaid           434298 non-null  float64       
 9   fee_charges_charged       434298 non-null  float64       
 10  fee_charges_repaid        434298 non-null  float64       
 11  penalty_charges_charged   434298 non-null  float64       
 12  pe

In [11]:
#drop all loan records that have irrelevant status keys OR were never disbursed
df = df.drop(df[
    (df["loan_status"] == 0) |
    (df["loan_status"] == 100) |
    (df["loan_status"] == 400) |
    (df["loan_status"] == 500)
].index)


#drop all rows where loan id is blank
df = df.loc[df["loan_mifos_id"].notnull()]


#drop all rows with duplicate loan id is blank
df = df.loc[~df["loan_id_product_concat"].duplicated()]


#drop all rows where disbursement date is blank
df = df.loc[df["disbursed_on_date"].notnull()]

In [12]:
#clean up mobile number column
df[["client_mobile_number_2", "temp"]] = df["client_mobile_number"].astype("str").str.split(".", expand=True)


#drop newly created temp column
df.drop(["temp","client_mobile_number"], axis=1, inplace=True)

df.rename(columns={"client_mobile_number_2": "client_mobile_number"}, inplace=True)

In [13]:
#sort dataframe based on specific columns
df.sort_values(["client_mobile_number","disbursed_on_date"], ascending=[True, False], inplace=True)


#create loan count column i.e adds a new column that captures the num of loans a customer has taken
df["loan_count"] = df.groupby("store_number")["store_number"].transform('size')

#create loan rank column i.e adds a new column that captures the rank of each loan a customer has taken
df["loan_rank"] = df.groupby("store_number")["loan_mifos_id"].rank(ascending=True)

##sort out minor Mifos errors relating to loans with status 700
df.loc[(df["loan_status"] == 700) & (df["closed_on_date"].isnull()), "closed_on_date"] = df["expected_matured_on_date"]

## Notes ==> fix issue where loan rank gets messed up because of different Mifos instance

In [14]:
#create due_date fixed column to ensure due date tallies with the term frequency for each loan

df["due_date_fixed"] = df["disbursed_on_date"] + pd.to_timedelta(df["term_frequency"], unit='d')

In [15]:
#create a copy of df

all_loans = df.copy()

In [16]:
#analysis period

print('analysis begin date {}'.format(all_loans['disbursed_on_date'].min()))
print('analysis latest date {}'.format(all_loans['disbursed_on_date'].max()))

analysis begin date 2017-12-10 00:00:00
analysis latest date 2022-09-22 00:00:00


In [17]:
def calc_days_past_due(df):
    """
    Function to calculate days past due for each loan record. 
    The function uses pandas.Series vectorized arguments to ensure fast iterations/loops.
    Inputs are the arguments passed in the conditions list.
    Outputs are the results that are reported based on the choices list.
    Each output/choice assigned corresponds to the input/conditions level assigned above i.e.
    the first condition corresponds to the first choice etc, as such the rows of conditions & choices need to match.
    If the output reported is a string '0', that's an error/edgecase whose conditions, choices were not well declared.
    
    Inputs: 
    1) The current status of a loan as captured on corebanking(Mifos),
    2) The loans due date, 
    3) the last repayment date on record for the loan.
    
    
    Outputs:
    A calculation of a loan's number of days past due, that is converted from datetime to integer.
    
    """
    #subtract one day from today timestamp to ensure evaluation is in-line with Mifos which is Time-1 i.e one day behind
    today = (pd.Timestamp.today()).strftime('%Y-%m-%d')
    today = pd.Timestamp(today) - dt.timedelta(days=1)
        
    loan_status = df["loan_status"]
    due_date = df["due_date_fixed"]
    closed_on_date = df["closed_on_date"]
    
    #the main input is the loan status id that is used to slice the df
    conditions = [
        loan_status.eq(300),
        loan_status.eq(600),
        loan_status.eq(601),
        loan_status.eq(700),
    ]
    
    #the other date inputs are used for calculation based on the condition of loan status of a loan at any given point
    choices = [
        (today - due_date).dt.days,
        (closed_on_date - due_date).dt.days,
        (closed_on_date - due_date).dt.days,
        (closed_on_date - due_date).dt.days,
    ]
    
    days_past_due = np.select(conditions, choices)
    
    return days_past_due


#apply the function to the df to create the days_past_due_column
all_loans["days_past_due"] = calc_days_past_due(all_loans)

In [18]:
def set_loan_status_labels(df):
    """
    Function to set the loan repayment status of a loan.
    Inputs are the arguments passed in the conditions list.
    Outputs are the results that are reported based on the choices list.
    Each output/choice assigned corresponds to the input/conditions level assigned above i.e.
    the first condition corresponds to the first choice etc, as such the rows of conditions & choices need to match.
    If the output reported is a string '0', that's an error/edgecase whose conditions, choices were not well declared.
    
    Inputs: 
    1) The current status of a loan as captured on corebanking(Mifos),
    2) Number of days past due for each loan, 
    3) Term frequency for each loan,
    4) The version of Bloom tied to the loan record
    
    Outputs: 
    A string label noting the current loan repayment status of each loan record
    """
    thirty_day_product_rollover = 7 #Bloom 1.0
    twenty_one_day_product_rollover = 5 #Bloom 2.0 only
    seven_day_product_rollover_bloom1 = 7 #Bloom 2.0 adjusted to 5 days from 7 days in Bloom 1.0
    seven_day_product_rollover_bloom2 = 5 
    three_day_product_rollover = 2
    one_day_product_rollover = 1 #Bloom 2.0, may be adjusted to 5 days as well

    loan_status = all_loans["loan_status"]
    days_past_due = all_loans["days_past_due"]
    term_frequency = all_loans["term_frequency"]
    bloom_version = all_loans["bloom_version"]

    conditions = [
        #written off loans
        loan_status.eq(601),
        
        #currently active OR loans closed in tenure
        loan_status.eq(300) & days_past_due.le(0),
        loan_status.eq(600) & days_past_due.lt(0),
        loan_status.eq(700) & days_past_due.lt(0),
        loan_status.eq(600) & days_past_due.eq(0),
        loan_status.eq(700) & days_past_due.eq(0),
    
        #active loans that are presently in rollover
        loan_status.eq(300) & term_frequency.eq(1) & days_past_due.le(one_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(3) & days_past_due.le(three_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.le(seven_day_product_rollover_bloom1),
        loan_status.eq(300) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.le(seven_day_product_rollover_bloom2),
        loan_status.eq(300) & term_frequency.eq(21) & days_past_due.le(twenty_one_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(30) & days_past_due.le(thirty_day_product_rollover),
    
        #loans that were cleared/closed after they got to rollover and cleared with exact balance due
        loan_status.eq(600) & term_frequency.eq(1) & days_past_due.le(one_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(3) & days_past_due.le(three_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.le(seven_day_product_rollover_bloom1),
        loan_status.eq(600) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.le(seven_day_product_rollover_bloom2),
        loan_status.eq(600) & term_frequency.eq(21) & days_past_due.le(twenty_one_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(30) & days_past_due.le(thirty_day_product_rollover),
    
        #loans that were cleared/closed after they got to rollover and were overpaid
        loan_status.eq(700) & term_frequency.eq(1) & days_past_due.le(one_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(3) & days_past_due.le(three_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.le(seven_day_product_rollover_bloom1),
        loan_status.eq(700) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.le(seven_day_product_rollover_bloom2),
        loan_status.eq(700) & term_frequency.eq(21) & days_past_due.le(twenty_one_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(30) & days_past_due.le(thirty_day_product_rollover),

        #active loans that are presently in default
        loan_status.eq(300) & term_frequency.eq(1) & days_past_due.gt(one_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(3) & days_past_due.gt(three_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.gt(seven_day_product_rollover_bloom1),
        loan_status.eq(300) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.gt(seven_day_product_rollover_bloom2),
        loan_status.eq(300) & term_frequency.eq(21) & days_past_due.gt(twenty_one_day_product_rollover),
        loan_status.eq(300) & term_frequency.eq(30) & days_past_due.gt(thirty_day_product_rollover),


        #loans were cleared/closed when they had got to default status and cleared with exact balance due
        loan_status.eq(600) & term_frequency.eq(1) & days_past_due.gt(one_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(3) & days_past_due.gt(three_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.gt(seven_day_product_rollover_bloom1),
        loan_status.eq(600) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.gt(seven_day_product_rollover_bloom2),
        loan_status.eq(600) & term_frequency.eq(21) & days_past_due.gt(twenty_one_day_product_rollover),
        loan_status.eq(600) & term_frequency.eq(30) & days_past_due.gt(thirty_day_product_rollover),

        #loans that were cleared/closed after they got to default and were overpaid
        loan_status.eq(700) & term_frequency.eq(1) & days_past_due.gt(one_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(3) & days_past_due.gt(three_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(7) & bloom_version.eq(1) & days_past_due.gt(seven_day_product_rollover_bloom1),
        loan_status.eq(700) & term_frequency.eq(7) & bloom_version.eq(2) & days_past_due.gt(seven_day_product_rollover_bloom2),
        loan_status.eq(700) & term_frequency.eq(21) & days_past_due.gt(twenty_one_day_product_rollover),
        loan_status.eq(700) & term_frequency.eq(30) & days_past_due.gt(thirty_day_product_rollover),
    ]


    choices = [
        "written-off_default",
        
        "current_active",
        "closed_early_repayment",
        "closed_early_repayment_overpaid",
        "closed_on_time",
        "closed_on_time_overpaid",    
        
        "active_rollover",
        "active_rollover",
        "active_rollover",
        "active_rollover",
        "active_rollover",
        "active_rollover",
        
        "closed_rollover",
        "closed_rollover",
        "closed_rollover",
        "closed_rollover",
        "closed_rollover",
        "closed_rollover",    
        
        "closed_rollover_overpaid",
        "closed_rollover_overpaid",
        "closed_rollover_overpaid",
        "closed_rollover_overpaid",
        "closed_rollover_overpaid",
        "closed_rollover_overpaid",
        
        "active_default",
        "active_default",
        "active_default",
        "active_default",
        "active_default",
        "active_default",   
        
        "closed_default",
        "closed_default",
        "closed_default",
        "closed_default",
        "closed_default",
        "closed_default",        
        
        "closed_default_overpaid",
        "closed_default_overpaid",
        "closed_default_overpaid",
        "closed_default_overpaid",
        "closed_default_overpaid",
        "closed_default_overpaid",

    ]
    
    loan_labels = np.select(conditions, choices)
    
    return loan_labels


#apply the function to the df to create the days_past_due_column
all_loans["loan_repayment_status"] = set_loan_status_labels(all_loans)

In [19]:
#return df with two most recent loans for each client id

#temp_df = all_loans.groupby("client_id").head(2).reset_index()

In [20]:
all_loans.head(2)

,client_mifos_id,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version,loan_id_product_concat,client_mobile_number,loan_count,loan_rank,due_date_fixed,days_past_due,loan_repayment_status
77620,53322,600,184191,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-08-03,2022-08-10,NaT,7761393.0,2.0,184191-2.0,254110007123,2.0,2.0,2022-08-10,NaN,0
76891,53322,600,161253,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-07-19,2022-07-26,2022-07-26,7761393.0,2.0,161253-2.0,254110007123,2.0,1.0,2022-07-26,0.0,closed_on_time


In [21]:
all_loans.shape

(361069, 25)

In [22]:
all_loans['store_number'].nunique()

48100

In [23]:
#convert col to string
all_loans["store_number"] = all_loans["store_number"].astype("str")


#clean up store number column
all_loans[["store_number", "temp"]] = all_loans["store_number"].astype("str").str.split(".", expand=True)


#drop newly created temp column
all_loans.drop(["temp",], axis=1, inplace=True)

In [24]:
all_loans.head(2)

,client_mifos_id,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version,loan_id_product_concat,client_mobile_number,loan_count,loan_rank,due_date_fixed,days_past_due,loan_repayment_status
77620,53322,600,184191,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-08-03,2022-08-10,NaT,7761393,2.0,184191-2.0,254110007123,2.0,2.0,2022-08-10,NaN,0
76891,53322,600,161253,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-07-19,2022-07-26,2022-07-26,7761393,2.0,161253-2.0,254110007123,2.0,1.0,2022-07-26,0.0,closed_on_time


In [25]:
all_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361069 entries, 77620 to 408091
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   client_mifos_id           361069 non-null  object        
 1   loan_status               361069 non-null  int64         
 2   loan_mifos_id             361069 non-null  int64         
 3   term_frequency            361069 non-null  int64         
 4   principal_disbursed       361069 non-null  float64       
 5   principal_repaid          361069 non-null  float64       
 6   interest_charged          361069 non-null  float64       
 7   interest_repaid           361069 non-null  float64       
 8   fee_charges_charged       361069 non-null  float64       
 9   fee_charges_repaid        361069 non-null  float64       
 10  penalty_charges_charged   361069 non-null  float64       
 11  penalty_charges_repaid    361069 non-null  float64       
 12

In [26]:
#all_loans= all_loans[all_loans['disbursed_on_date'] <= '2022-06-14']

---
#### Generate aggregate summaries

In [27]:
#return df with the most recent loan for each borrower

agg_summary = all_loans.groupby("store_number").head(1).reset_index()

In [28]:
agg_summary.shape

(48101, 26)

In [29]:
agg_summary['store_number'].nunique()

48101

In [30]:
#trim df to only relevant columns
target_cols = ["client_mobile_number","store_number","loan_count","loan_status","term_frequency",
  "principal_disbursed","principal_repaid","disbursed_on_date",
  "expected_matured_on_date","closed_on_date","due_date_fixed",
  "days_past_due","bloom_version","loan_repayment_status"]

agg_summary = agg_summary[target_cols]

In [31]:
#aggregate maximum principal disbursed for each client id

agg_summary = pd.merge(agg_summary, (all_loans.groupby("store_number")["principal_disbursed"].max().rename("max_principal_amount").reset_index()), on="store_number")

In [32]:
#get df for when a customer got their max loan principal

max_principal_dates = all_loans.sort_values("principal_disbursed", ascending=False).groupby("store_number").first().reset_index()

In [33]:
#trim df to only remain with relevant columns

max_principal_dates = max_principal_dates[["store_number","disbursed_on_date"]]

#rename column to make it clearer
max_principal_dates.rename(columns = {"disbursed_on_date": "max_loan_disbursement_date"}, inplace = True)

#merge df

agg_summary = pd.merge(agg_summary, max_principal_dates, on="store_number")

In [34]:
#aggregate of loans well paid or in good standing

agg_good_loans = (all_loans.loc[(all_loans["loan_repayment_status"]== "closed_early_repayment")|
                                (all_loans["loan_repayment_status"]== "closed_early_repayment_overpaid")|
                                (all_loans["loan_repayment_status"]== "closed_on_time")|
                                (all_loans["loan_repayment_status"]== "closed_on_time_overpaid")|
                                (all_loans["loan_repayment_status"]== "current_active")
                               ]).groupby("store_number")["loan_id_product_concat"].aggregate("count").rename("count_good_loans").reset_index()

In [35]:
#merge agg_summary & agg good loans

agg_summary = pd.merge(agg_summary, agg_good_loans, how="outer", on="store_number")

In [36]:
#fill rest of missing values with zeros

cols_fillna = ['count_good_loans']
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    agg_summary[col].fillna(0,inplace=True)

In [37]:
#calculate good repayment ratios for the borrowers

agg_summary["good_loans_repayment_ratio"] = round(agg_summary["count_good_loans"]/agg_summary["loan_count"], 2)

In [38]:
#calculate num days since last disbursement
#subtract one day from today timestamp to ensure evaluation is in-line with Mifos which is Time-1 i.e one day behind

today = (pd.Timestamp.today()).strftime('%Y-%m-%d')
today = pd.Timestamp(today) - dt.timedelta(days=1)

agg_summary["num_days_since_last_disbursement"] = pd.to_numeric((today - agg_summary["disbursed_on_date"]).dt.days, downcast='integer')

In [39]:
#delete test accounts

agg_summary = agg_summary[agg_summary["max_principal_amount"]>=200]

In [40]:
#delete customers without a store number

agg_summary = agg_summary.loc[agg_summary["store_number"].notnull()]

In [41]:
#add column for max dpd

agg_summary = pd.merge(agg_summary, (all_loans.groupby("store_number")["days_past_due"].max().rename("max_days_past_due").reset_index()), on="store_number")

In [42]:
agg_summary.head(2)

,client_mobile_number,store_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,max_days_past_due
0,254110007123,7761393,2.0,600,7,1500.0,1500.0,2022-08-03,2022-08-10,NaT,2022-08-10,NaN,2.0,0,1500.0,2022-08-03,1.0,0.50,49,0.0
1,254110013557,7767671,6.0,300,21,14400.0,0.0,2022-06-09,2022-06-30,NaT,2022-06-30,83.0,2.0,active_default,14400.0,2022-06-07,4.0,0.67,104,83.0


In [43]:
agg_summary.shape

(48066, 20)

---
#### slice of DF with customers who have a max 30DPD

In [44]:
#get a slice of df where customers have a max dpd of 30 days past due

max_dpd_30 = agg_summary.loc[agg_summary["max_days_past_due"]<=30]

In [45]:
max_dpd_30.head(3)

,client_mobile_number,store_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,max_days_past_due
0,254110007123,7761393,2.0,600,7,1500.0,1500.0,2022-08-03,2022-08-10,NaT,2022-08-10,NaN,2.0,0,1500.0,2022-08-03,1.0,0.50,49,0.0
2,254110023683,7904403,1.0,300,7,2300.0,0.0,2022-09-04,2022-09-11,NaT,2022-09-11,10.0,2.0,active_default,2300.0,2022-09-04,0.0,0.00,17,10.0
3,254110032116,7414322,11.0,600,1,600.0,600.0,2022-08-08,2022-08-09,NaT,2022-08-09,NaN,2.0,0,600.0,2022-07-22,8.0,0.73,44,0.0


In [46]:
max_dpd_30_list = list(max_dpd_30["store_number"].unique())

In [47]:
len(max_dpd_30_list)

15719

---
#### get slice of df for loans over the past 3 months i.e 180 days

In [48]:
#filter df for 6+ loans & past 90 days

current_period = all_loans["disbursed_on_date"].max()
target_analysis_period = current_period - pd.DateOffset(months=3)
min_loan_amount = 200

loans_past_3_months = all_loans.loc[(all_loans["disbursed_on_date"]>=target_analysis_period)&
                                    (all_loans["principal_disbursed"]>=min_loan_amount)
                                   ]

In [49]:
all_loans.head(3)

,client_mifos_id,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version,loan_id_product_concat,client_mobile_number,loan_count,loan_rank,due_date_fixed,days_past_due,loan_repayment_status
77620,53322,600,184191,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.00,0.0,0.00,2022-08-03,2022-08-10,NaT,7761393,2.0,184191-2.0,254110007123,2.0,2.0,2022-08-10,NaN,0
76891,53322,600,161253,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.00,0.0,0.00,2022-07-19,2022-07-26,2022-07-26,7761393,2.0,161253-2.0,254110007123,2.0,1.0,2022-07-26,0.0,closed_on_time
431263,11011,300,118239,21,14400.0,0.0,1094.41,0.00,237.6,0.0,6349.07,0.0,22081.08,2022-06-09,2022-06-30,NaT,7767671,2.0,118239-2.0,254110013557,6.0,6.0,2022-06-30,83.0,active_default


In [50]:
all_loans.shape

(361069, 25)

In [51]:
#drop rows where store number is null

loans_past_3_months = loans_past_3_months.loc[loans_past_3_months["store_number"].notnull()]

In [52]:
#calculate num of loans taken past 3 months

loans_past_3_months["loan_count_past_3_months"] = loans_past_3_months.groupby("store_number")["store_number"].transform('size')

In [53]:
#create loan count column i.e adds a new column that captures the num of 7-day loans a customer has taken
loans_past_3_months["count_7_day_loans"] = loans_past_3_months.loc[loans_past_3_months["term_frequency"]==7].groupby("store_number")["store_number"].transform('size')

In [54]:
#fill rest of missing values with zeros

cols_fillna = ["count_7_day_loans"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months[col].fillna(0,inplace=True)

In [55]:
#how many customers have taken a loan over the past

loans_past_3_months["store_number"].nunique()

17300

In [56]:
loans_past_3_months.head(2)

,client_mifos_id,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version,loan_id_product_concat,client_mobile_number,loan_count,loan_rank,due_date_fixed,days_past_due,loan_repayment_status,loan_count_past_3_months,count_7_day_loans
77620,53322,600,184191,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-08-03,2022-08-10,NaT,7761393,2.0,184191-2.0,254110007123,2.0,2.0,2022-08-10,NaN,0,2,2.0
76891,53322,600,161253,7,1500.0,1500.0,39.01,39.01,0.0,0.0,0.0,0.0,0.0,2022-07-19,2022-07-26,2022-07-26,7761393,2.0,161253-2.0,254110007123,2.0,1.0,2022-07-26,0.0,closed_on_time,2,2.0


In [57]:
loans_past_3_months_stores = loans_past_3_months.groupby(['store_number'], as_index=False)['loan_count_past_3_months'].max()
loans_past_3_months_stores.head(3)

,store_number,loan_count_past_3_months
0,110501,3
1,110901,4
2,110929,2


In [58]:
loans_past_3_months_stores.shape

(17300, 2)

In [59]:
#get list of all store numbers

loans_past_3_months_list = list(loans_past_3_months["store_number"].unique())

---
##### get agg summaries for past 3 months 7-day loans summaries

In [60]:
#get aggregate summary with avg loan tenure

loans_past_3_months_summary = round(loans_past_3_months.groupby("store_number")["term_frequency"].aggregate("mean").rename("avg_loan_tenure").reset_index(), 0)

In [61]:
#get summary of number of 7 day loans a customer has taken

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, loans_past_3_months.groupby("store_number")["count_7_day_loans"].max().rename("count_7_day_loans").reset_index(), how="left", on="store_number")

In [62]:
#merge df to get number of 7 day loans paid upto rollover

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, (loans_past_3_months.loc[loans_past_3_months["term_frequency"]==7].loc[
    (loans_past_3_months["loan_repayment_status"]== "closed_early_repayment")|
    (loans_past_3_months["loan_repayment_status"]== "closed_early_repayment_overpaid")|
    (loans_past_3_months["loan_repayment_status"]== "closed_on_time")|
    (loans_past_3_months["loan_repayment_status"]== "closed_on_time_overpaid")|
    (loans_past_3_months["loan_repayment_status"]== "current_active")|
    (loans_past_3_months["loan_repayment_status"]== "active_rollover")|
    (loans_past_3_months["loan_repayment_status"]== "closed_rollover")|
    (loans_past_3_months["loan_repayment_status"]== "closed_rollover_overpaid")
]).groupby("store_number")["client_mifos_id"].aggregate("count").rename("count_7_day_loans_paid_upto_rollover").reset_index(), how="left", on="store_number")

In [63]:
#fill rest of missing values with zeros

cols_fillna = ["count_7_day_loans_paid_upto_rollover"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [64]:
#calculate good repayment ratios for the borrowers

loans_past_3_months_summary["good_loans_repayment_ratio(7_day_loans)"] = round(loans_past_3_months_summary["count_7_day_loans_paid_upto_rollover"]/loans_past_3_months_summary["count_7_day_loans"], 2)

In [65]:
#fill rest of missing values with zeros

cols_fillna = ["good_loans_repayment_ratio(7_day_loans)"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [66]:
# get minimum amount disbursed for a 7 day loan

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, loans_past_3_months.loc[loans_past_3_months["term_frequency"]==7].groupby("store_number")["principal_disbursed"].min().rename("minimum_7_day_principal_disbursed").reset_index(), how="left", on="store_number")

In [67]:
# get total amount disbursed for a 7 day loan

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, loans_past_3_months.loc[loans_past_3_months["term_frequency"]==7].groupby("store_number")["principal_disbursed"].sum().rename("total_sum_7_day_principal_disbursed").reset_index(), how="left", on="store_number")

In [68]:
# get total amount disbursed for a 1 day loan

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, loans_past_3_months.loc[loans_past_3_months["term_frequency"]==1].groupby("store_number")["principal_disbursed"].sum().rename("total_sum_1_day_principal_disbursed").reset_index(), how="left", on="store_number")

In [69]:
#fill rest of missing values with zeros

cols_fillna = ["minimum_7_day_principal_disbursed"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [70]:
#fill rest of missing values with zeros

cols_fillna = ["total_sum_7_day_principal_disbursed"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [71]:
#fill rest of missing values with zeros

cols_fillna = ["total_sum_1_day_principal_disbursed"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [72]:
#get the average 7 day loan principal ==> proxy for 7 day limit

loans_past_3_months_summary = pd.merge(loans_past_3_months_summary, loans_past_3_months.loc[loans_past_3_months["term_frequency"]==7].groupby("store_number")["principal_disbursed"].mean().rename("avg_7_day_principal_disbursed").reset_index(), how="left", on="store_number")

In [73]:
#fill rest of missing values with zeros

cols_fillna = ["avg_7_day_principal_disbursed"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    loans_past_3_months_summary[col].fillna(0,inplace=True)

In [74]:
def approximate_21_day_limit(df):
    """
    """
    
    seven_day_limit = df["avg_7_day_principal_disbursed"]
    seven_day_limit_factor = 0.17
    twenty_one_day_limit_factor = 0.5
    product_cap = 200000
    zero = 0
    
    operation = (np.ceil(((df["avg_7_day_principal_disbursed"]*twenty_one_day_limit_factor)/seven_day_limit_factor)/100) * 100).astype(int)
    
    conditions = [
        seven_day_limit.eq(zero),
        operation.le(product_cap),
        operation.gt(product_cap),
    ]
    
    choices = [
        zero,
        operation,
        product_cap,
    ]
    
    limit_col = np.select(conditions, choices)
    
    return limit_col

#execute function

loans_past_3_months_summary["21_day_limit"] = approximate_21_day_limit(loans_past_3_months_summary)

In [75]:
loans_past_3_months_summary.shape

(17300, 10)

In [76]:
loans_past_3_months_summary.head()

,store_number,avg_loan_tenure,count_7_day_loans,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,21_day_limit
0,110501,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,110901,18.0,1.0,1.0,1.0,36000.0,36000.0,0.0,36000.0,105900
2,110929,7.0,2.0,2.0,1.0,1600.0,3200.0,0.0,1600.0,4800
3,110970,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,111160,16.0,1.0,1.0,1.0,8000.0,8000.0,0.0,8000.0,23600


In [77]:
#### num of customers with loans repayment ratio of >=90% for their 7 day loans over the past 3 months

loans_past_3_months_summary.loc[loans_past_3_months_summary["good_loans_repayment_ratio(7_day_loans)"]>=0.9]["store_number"].nunique()

5606

In [78]:
# num of customers with total 1 day sum of principal > 35K

loans_past_3_months_summary.loc[loans_past_3_months_summary["total_sum_7_day_principal_disbursed"]>=35000]["store_number"].nunique()

3334

In [79]:
loans_past_3_months_summary.loc[loans_past_3_months_summary["count_7_day_loans"]>=3]["store_number"].nunique()

6166

In [80]:
# num of customers with 21 day exceeding 7500

loans_past_3_months_summary.loc[loans_past_3_months_summary["21_day_limit"]>=7500]["store_number"].nunique()

7953

In [81]:
# num of customers with an average loan tenure greater than 1

loans_past_3_months_summary.loc[loans_past_3_months_summary["avg_loan_tenure"]>1]["store_number"].nunique()

16174

In [82]:
loans_past_3_months_summary.head()

,store_number,avg_loan_tenure,count_7_day_loans,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,21_day_limit
0,110501,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,110901,18.0,1.0,1.0,1.0,36000.0,36000.0,0.0,36000.0,105900
2,110929,7.0,2.0,2.0,1.0,1600.0,3200.0,0.0,1600.0,4800
3,110970,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,111160,16.0,1.0,1.0,1.0,8000.0,8000.0,0.0,8000.0,23600


In [83]:
loans_past_3_months_summary.head(2)

,store_number,avg_loan_tenure,count_7_day_loans,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,21_day_limit
0,110501,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,110901,18.0,1.0,1.0,1.0,36000.0,36000.0,0.0,36000.0,105900


In [84]:
loans_past_3_months_summary['total_sum_7_day_principal_disbursed'].max()

3200000.0

In [85]:
loans_past_3_months_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17300 entries, 0 to 17299
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   store_number                             17300 non-null  object 
 1   avg_loan_tenure                          17300 non-null  float64
 2   count_7_day_loans                        17300 non-null  float64
 3   count_7_day_loans_paid_upto_rollover     17300 non-null  float64
 4   good_loans_repayment_ratio(7_day_loans)  17300 non-null  float64
 5   minimum_7_day_principal_disbursed        17300 non-null  float64
 6   total_sum_7_day_principal_disbursed      17300 non-null  float64
 7   total_sum_1_day_principal_disbursed      17300 non-null  float64
 8   avg_7_day_principal_disbursed            17300 non-null  float64
 9   21_day_limit                             17300 non-null  int64  
dtypes: float64(8), int64(1), object(1)
memory usag

---
#### Fetch latest limits data from warehouse

In [86]:
df = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220922_multiple_products_2.xlsx")

df = df[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]

df.shape

(60233, 4)

In [87]:
df[['final_21_limit', 'final_7_limit', 'final_1_limit']].sum()

final_21_limit    353222300
final_7_limit     480198200
final_1_limit     482168800
dtype: int64

In [88]:
#replace the in the column headers with underscores & make all column headers in lower case
df.columns = df.columns.str.lower().str.replace(' ','')

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60233 entries, 0 to 60232
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   store_number    60233 non-null  int64
 1   final_21_limit  60233 non-null  int64
 2   final_7_limit   60233 non-null  int64
 3   final_1_limit   60233 non-null  int64
dtypes: int64(4)
memory usage: 1.8 MB


In [90]:
#convert specific string columns to float, first by leaving out the non-targeted columns
string_to_float_cols = df.columns.drop(["final_21_limit", "final_7_limit", "final_1_limit"])

df[string_to_float_cols] = df[string_to_float_cols].apply(pd.to_numeric, errors='coerce')

In [91]:
df.shape

(60233, 4)

In [92]:
## get slice of df where customer has 7 day limit but 21 day limit is 0

#slice_df = df.loc[(df["final_7_limit"]>200)&(df["final_21_limit"]==0)]

slice_df = df.loc[df["final_21_limit"]>0]

In [93]:
slice_df.shape

(5522, 4)

In [94]:
#convert store num col to str

slice_df["store_number"] = slice_df["store_number"].astype("str")

/tmp/ipykernel_154845/3357166545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slice_df["store_number"] = slice_df["store_number"].astype("str")


In [95]:
## num customers that have 7 day limit but 21 day limit is 0

slice_df["store_number"].nunique()

5522

In [96]:
#get list of all target store numbers

slice_df_list = list(slice_df["store_number"].unique())

In [97]:
len(slice_df_list)

5522

In [98]:
#df["store_number"].nunique()

In [99]:
df_list = list(df["store_number"].unique())

In [100]:
len(df_list)

60233

---
#### Applying all factors to get the final df

In [101]:
#filter loans summary df to see how many customers had 7 day limit but no 21 day limit

target_customers_scope = loans_past_3_months_summary.loc[loans_past_3_months_summary["store_number"].isin(slice_df_list)]

print(target_customers_scope.shape)

(5250, 10)


In [102]:
target_customers_scope.head(3)

,store_number,avg_loan_tenure,count_7_day_loans,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,21_day_limit
0,110501,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,110901,18.0,1.0,1.0,1.0,36000.0,36000.0,0.0,36000.0,105900
3,110970,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [103]:
#check to make sure customer has no history of >30 DPD

ntarget_customers_scope = target_customers_scope.loc[target_customers_scope["store_number"].isin(max_dpd_30_list)]

print(ntarget_customers_scope.shape)

(3388, 10)


In [104]:
ntarget_customers_scope.loc[(ntarget_customers_scope["good_loans_repayment_ratio(7_day_loans)"]>=0.9)].shape

(1529, 10)

In [105]:
ntarget_customers_scope.loc[(ntarget_customers_scope["total_sum_7_day_principal_disbursed"]>=35000)].shape

(830, 10)

In [106]:
ntarget_customers_scope.loc[(ntarget_customers_scope["count_7_day_loans"]>=3)].shape

(1162, 10)

In [107]:
ntarget_customers_scope.loc[(ntarget_customers_scope["21_day_limit"]>=7500)].shape

(1649, 10)

In [108]:
#all filters applied

mtarget_customers_scope = ntarget_customers_scope.loc[
    (ntarget_customers_scope["good_loans_repayment_ratio(7_day_loans)"]>=0.9)&
    (ntarget_customers_scope["21_day_limit"]>=7500)
]

print(mtarget_customers_scope.shape)

(1299, 10)


In [109]:
mtarget_customers_scope.shape

(1299, 10)

In [110]:
mtarget_customers_scope.head(3)

,store_number,avg_loan_tenure,count_7_day_loans,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,21_day_limit
7,112171,10.0,12.0,12.0,1.0,1200.0,106300.0,0.0,8858.333333,26100
17,116716,7.0,7.0,7.0,1.0,8000.0,56000.0,0.0,8000.000000,23600
23,118081,14.0,1.0,1.0,1.0,5000.0,5000.0,3000.0,5000.000000,14800


In [111]:
qualified_stores = mtarget_customers_scope[['store_number']]

df['store_number'] = df['store_number'].astype(str)

final = pd.merge(qualified_stores, df, on = 'store_number', how = 'left')

final.head()

,store_number,final_21_limit,final_7_limit,final_1_limit
0,112171,21700,14000,14000
1,116716,34300,19800,19800
2,118081,53200,33300,33300
3,124149,17700,10700,10700
4,125277,131400,73000,73000


In [112]:
final.shape

(1299, 4)

In [113]:
final['final_21_limit'].sum()

87898500

In [114]:
final_21_day = final[['store_number', 'final_21_limit']]

final_21_day.head()

,store_number,final_21_limit
0,112171,21700
1,116716,34300
2,118081,53200
3,124149,17700
4,125277,131400


In [115]:
df_adj = df[['store_number', 'final_7_limit', 'final_1_limit']]

df_adj.head()

,store_number,final_7_limit,final_1_limit
0,786152,0,0
1,165978,0,0
2,745142,0,0
3,978161,0,0
4,510261,0,0


In [116]:
final_limits = pd.merge(final_21_day, df_adj, on = 'store_number', how = 'outer')

final_limits.head()

,store_number,final_21_limit,final_7_limit,final_1_limit
0,112171,21700.0,14000,14000
1,116716,34300.0,19800,19800
2,118081,53200.0,33300,33300
3,124149,17700.0,10700,10700
4,125277,131400.0,73000,73000


In [117]:
final_limits.shape

(60233, 4)

In [118]:
final_limits[['final_21_limit', 'final_7_limit', 'final_1_limit']].sum()

final_21_limit     87898500.0
final_7_limit     480198200.0
final_1_limit     482168800.0
dtype: float64

In [119]:
final_limits.to_excel('final_limits_new.xlsx')